In [1]:
seed = 42  # for reproducibility

# Imports
import os
import sys
import enum
import copy
import random
import tempfile
import warnings
import multiprocessing
import numpy as np
import pandas as pd
import nibabel as nib
import SimpleITK as sitk
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython import display

# import visdom
from pathlib import Path
from tqdm import tqdm
from scipy import stats

import torch
import torch.nn.functional as F
from torchvision.utils import make_grid, save_image
torch.manual_seed(seed)
import torchio as tio
sys.path.append(str(Path().absolute().parent))

If you use TorchIO for your research, please cite the following paper:
Pérez-García et al., TorchIO: a Python library for efficient loading,
preprocessing, augmentation and patch-based sampling of medical images
in deep learning. Credits instructions: https://torchio.readthedocs.io/#credits



In [2]:
DDSM_mass_path = Path('/home/linardos/Datasets/DDSM/Images/CBIS-DDSM/Mass')
train_path = DDSM_mass_path.joinpath('Train')

In [3]:
csv_file = pd.read_csv(Path('/home/linardos/Datasets/DDSM/calc_case_description_test_set.csv'))
new_csv_file = pd.read_csv(Path('/home/linardos/Datasets/DDSM/calc_case_description_test_set.csv'))

In [4]:
csv_file

,patient_id,breast density,left or right breast,image view,abnormality id,abnormality type,calc type,calc distribution,assessment,pathology,subtlety,image file path,cropped image file path,ROI mask file path
0,P_00038,2,LEFT,CC,1,calcification,PUNCTATE-PLEOMORPHIC,CLUSTERED,4,BENIGN,2,Calc-Test_P_00038_LEFT_CC/1.3.6.1.4.1.9590.100...,Calc-Test_P_00038_LEFT_CC_1/1.3.6.1.4.1.9590.1...,Calc-Test_P_00038_LEFT_CC_1/1.3.6.1.4.1.9590.1...
1,P_00038,2,LEFT,MLO,1,calcification,PUNCTATE-PLEOMORPHIC,CLUSTERED,4,BENIGN,2,Calc-Test_P_00038_LEFT_MLO/1.3.6.1.4.1.9590.10...,Calc-Test_P_00038_LEFT_MLO_1/1.3.6.1.4.1.9590....,Calc-Test_P_00038_LEFT_MLO_1/1.3.6.1.4.1.9590....
2,P_00038,2,RIGHT,CC,1,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,Calc-Test_P_00038_RIGHT_CC/1.3.6.1.4.1.9590.10...,Calc-Test_P_00038_RIGHT_CC_1/1.3.6.1.4.1.9590....,Calc-Test_P_00038_RIGHT_CC_1/1.3.6.1.4.1.9590....
3,P_00038,2,RIGHT,CC,2,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,Calc-Test_P_00038_RIGHT_CC/1.3.6.1.4.1.9590.10...,Calc-Test_P_00038_RIGHT_CC_2/1.3.6.1.4.1.9590....,Calc-Test_P_00038_RIGHT_CC_2/1.3.6.1.4.1.9590....
4,P_00038,2,RIGHT,MLO,1,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,Calc-Test_P_00038_RIGHT_MLO/1.3.6.1.4.1.9590.1...,Calc-Test_P_00038_RIGHT_MLO_1/1.3.6.1.4.1.9590...,Calc-Test_P_00038_RIGHT_MLO_1/1.3.6.1.4.1.9590...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,P_02464,2,RIGHT,MLO,1,calcification,FINE_LINEAR_BRANCHING,CLUSTERED,0,MALIGNANT,4,Calc-Test_P_02464_RIGHT_MLO/1.3.6.1.4.1.9590.1...,Calc-Test_P_02464_RIGHT_MLO_1/1.3.6.1.4.1.9590...,Calc-Test_P_02464_RIGHT_MLO_1/1.3.6.1.4.1.9590...
322,P_02498,4,RIGHT,CC,1,calcification,PUNCTATE,CLUSTERED,0,BENIGN,3,Calc-Test_P_02498_RIGHT_CC/1.3.6.1.4.1.9590.10...,Calc-Test_P_02498_RIGHT_CC_1/1.3.6.1.4.1.9590....,Calc-Test_P_02498_RIGHT_CC_1/1.3.6.1.4.1.9590....
323,P_02498,4,RIGHT,MLO,1,calcification,PUNCTATE,CLUSTERED,0,BENIGN,3,Calc-Test_P_02498_RIGHT_MLO/1.3.6.1.4.1.9590.1...,Calc-Test_P_02498_RIGHT_MLO_1/1.3.6.1.4.1.9590...,Calc-Test_P_02498_RIGHT_MLO_1/1.3.6.1.4.1.9590...
324,P_02501,3,RIGHT,CC,1,calcification,PLEOMORPHIC,CLUSTERED,0,MALIGNANT,3,Calc-Test_P_02501_RIGHT_CC/1.3.6.1.4.1.9590.10...,Calc-Test_P_02501_RIGHT_CC_1/1.3.6.1.4.1.9590....,Calc-Test_P_02501_RIGHT_CC_1/1.3.6.1.4.1.9590....


In [5]:
csv_file['image file path'][0]

'Calc-Test_P_00038_LEFT_CC/1.3.6.1.4.1.9590.100.1.2.85935434310203356712688695661986996009/1.3.6.1.4.1.9590.100.1.2.374115997511889073021386151921807063992/000000.dcm'

In [6]:
csv_file['image file path'][5]

'Calc-Test_P_00038_RIGHT_MLO/1.3.6.1.4.1.9590.100.1.2.328421320411501709324953698601549885215/1.3.6.1.4.1.9590.100.1.2.44262460211112930513355519060642708846/000000.dcm'

In [7]:
new_csv_file['image file path'] = csv_file['image file path'].str.replace('Calc-Test_','')
new_csv_file['image file path'] = new_csv_file['image file path'].str.split('/', expand=True)[0]
new_csv_file['image file path'] = 'Test/' + new_csv_file['image file path'].astype(str) + '/scan.nii.gz'
new_csv_file['cropped image file path'] = csv_file['cropped image file path'].str.replace('Calc-Test_','')
new_csv_file['cropped image file path'] = new_csv_file['cropped image file path'].str.split('/', expand=True)[0]
new_csv_file['cropped image file path'] = 'Test/' + new_csv_file['cropped image file path'].astype(str) + '/roi_calc.nii.gz'
new_csv_file['cropped image file path'] = new_csv_file['cropped image file path'].str.replace('_1/r','/r')
new_csv_file['ROI mask file path'] = csv_file['ROI mask file path'].str.replace('Calc-Test_','')
new_csv_file['ROI mask file path'] = new_csv_file['ROI mask file path'].str.split('/', expand=True)[0]
new_csv_file['ROI mask file path'] = 'Test/' + new_csv_file['ROI mask file path'].astype(str) + '/mask_calc.nii.gz'
new_csv_file['ROI mask file path'] = new_csv_file['ROI mask file path'].str.replace('_1/m','/m')

new_csv_file['image file path'] = new_csv_file['image file path'].str.replace('_LEFT','/LEFT')
new_csv_file['image file path'] = new_csv_file['image file path'].str.replace('_RIGHT','/RIGHT')

new_csv_file['cropped image file path'] = new_csv_file['cropped image file path'].str.replace('_LEFT','/LEFT')
new_csv_file['cropped image file path'] = new_csv_file['cropped image file path'].str.replace('_RIGHT','/RIGHT')

new_csv_file['ROI mask file path'] = new_csv_file['ROI mask file path'].str.replace('_LEFT','/LEFT')
new_csv_file['ROI mask file path'] = new_csv_file['ROI mask file path'].str.replace('_RIGHT','/RIGHT')



In [8]:
new_csv_file['cropped image file path'] = new_csv_file['cropped image file path'].str.replace(f'CC/roi_calc.nii',f'CC/roi_calc_1.nii')
new_csv_file['cropped image file path'] = new_csv_file['cropped image file path'].str.replace(f'CC/mask_calc.nii',f'CC/mask_calc_1.nii')
new_csv_file['cropped image file path'] = new_csv_file['cropped image file path'].str.replace(f'MLO/roi_calc.nii',f'MLO/roi_calc_1.nii')
new_csv_file['cropped image file path'] = new_csv_file['cropped image file path'].str.replace(f'MLO/mask_calc.nii',f'MLO/mask_calc_1.nii')

new_csv_file['ROI mask file path'] = new_csv_file['ROI mask file path'].str.replace(f'CC/roi_calc.nii',f'CC/roi_calc_1.nii')
new_csv_file['ROI mask file path'] = new_csv_file['ROI mask file path'].str.replace(f'CC/mask_calc.nii',f'CC/mask_calc_1.nii')
new_csv_file['ROI mask file path'] = new_csv_file['ROI mask file path'].str.replace(f'MLO/roi_calc.nii',f'MLO/roi_calc_1.nii')
new_csv_file['ROI mask file path'] = new_csv_file['ROI mask file path'].str.replace(f'MLO/mask_calc.nii',f'MLO/mask_calc_1.nii')

for i in range(20):
    new_csv_file['cropped image file path'] = new_csv_file['cropped image file path'].str.replace(f'_{i}/roi_calc.nii',f'/roi_calc_{i}.nii')
    new_csv_file['ROI mask file path'] = new_csv_file['ROI mask file path'].str.replace(f'_{i}/mask_calc.nii',f'/mask_calc_{i}.nii')

new_csv_file.columns = new_csv_file.columns.str.replace(' ','_')

In [9]:
new_csv_file

,patient_id,breast_density,left_or_right_breast,image_view,abnormality_id,abnormality_type,calc_type,calc_distribution,assessment,pathology,subtlety,image_file_path,cropped_image_file_path,ROI_mask_file_path
0,P_00038,2,LEFT,CC,1,calcification,PUNCTATE-PLEOMORPHIC,CLUSTERED,4,BENIGN,2,Test/P_00038/LEFT_CC/scan.nii.gz,Test/P_00038/LEFT_CC/roi_calc_1.nii.gz,Test/P_00038/LEFT_CC/mask_calc_1.nii.gz
1,P_00038,2,LEFT,MLO,1,calcification,PUNCTATE-PLEOMORPHIC,CLUSTERED,4,BENIGN,2,Test/P_00038/LEFT_MLO/scan.nii.gz,Test/P_00038/LEFT_MLO/roi_calc_1.nii.gz,Test/P_00038/LEFT_MLO/mask_calc_1.nii.gz
2,P_00038,2,RIGHT,CC,1,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,Test/P_00038/RIGHT_CC/scan.nii.gz,Test/P_00038/RIGHT_CC/roi_calc_1.nii.gz,Test/P_00038/RIGHT_CC/mask_calc_1.nii.gz
3,P_00038,2,RIGHT,CC,2,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,Test/P_00038/RIGHT_CC/scan.nii.gz,Test/P_00038/RIGHT_CC/roi_calc_2.nii.gz,Test/P_00038/RIGHT_CC/mask_calc_2.nii.gz
4,P_00038,2,RIGHT,MLO,1,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,Test/P_00038/RIGHT_MLO/scan.nii.gz,Test/P_00038/RIGHT_MLO/roi_calc_1.nii.gz,Test/P_00038/RIGHT_MLO/mask_calc_1.nii.gz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,P_02464,2,RIGHT,MLO,1,calcification,FINE_LINEAR_BRANCHING,CLUSTERED,0,MALIGNANT,4,Test/P_02464/RIGHT_MLO/scan.nii.gz,Test/P_02464/RIGHT_MLO/roi_calc_1.nii.gz,Test/P_02464/RIGHT_MLO/mask_calc_1.nii.gz
322,P_02498,4,RIGHT,CC,1,calcification,PUNCTATE,CLUSTERED,0,BENIGN,3,Test/P_02498/RIGHT_CC/scan.nii.gz,Test/P_02498/RIGHT_CC/roi_calc_1.nii.gz,Test/P_02498/RIGHT_CC/mask_calc_1.nii.gz
323,P_02498,4,RIGHT,MLO,1,calcification,PUNCTATE,CLUSTERED,0,BENIGN,3,Test/P_02498/RIGHT_MLO/scan.nii.gz,Test/P_02498/RIGHT_MLO/roi_calc_1.nii.gz,Test/P_02498/RIGHT_MLO/mask_calc_1.nii.gz
324,P_02501,3,RIGHT,CC,1,calcification,PLEOMORPHIC,CLUSTERED,0,MALIGNANT,3,Test/P_02501/RIGHT_CC/scan.nii.gz,Test/P_02501/RIGHT_CC/roi_calc_1.nii.gz,Test/P_02501/RIGHT_CC/mask_calc_1.nii.gz


In [10]:
new_csv_file

,patient_id,breast_density,left_or_right_breast,image_view,abnormality_id,abnormality_type,calc_type,calc_distribution,assessment,pathology,subtlety,image_file_path,cropped_image_file_path,ROI_mask_file_path
0,P_00038,2,LEFT,CC,1,calcification,PUNCTATE-PLEOMORPHIC,CLUSTERED,4,BENIGN,2,Test/P_00038/LEFT_CC/scan.nii.gz,Test/P_00038/LEFT_CC/roi_calc_1.nii.gz,Test/P_00038/LEFT_CC/mask_calc_1.nii.gz
1,P_00038,2,LEFT,MLO,1,calcification,PUNCTATE-PLEOMORPHIC,CLUSTERED,4,BENIGN,2,Test/P_00038/LEFT_MLO/scan.nii.gz,Test/P_00038/LEFT_MLO/roi_calc_1.nii.gz,Test/P_00038/LEFT_MLO/mask_calc_1.nii.gz
2,P_00038,2,RIGHT,CC,1,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,Test/P_00038/RIGHT_CC/scan.nii.gz,Test/P_00038/RIGHT_CC/roi_calc_1.nii.gz,Test/P_00038/RIGHT_CC/mask_calc_1.nii.gz
3,P_00038,2,RIGHT,CC,2,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,Test/P_00038/RIGHT_CC/scan.nii.gz,Test/P_00038/RIGHT_CC/roi_calc_2.nii.gz,Test/P_00038/RIGHT_CC/mask_calc_2.nii.gz
4,P_00038,2,RIGHT,MLO,1,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,Test/P_00038/RIGHT_MLO/scan.nii.gz,Test/P_00038/RIGHT_MLO/roi_calc_1.nii.gz,Test/P_00038/RIGHT_MLO/mask_calc_1.nii.gz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,P_02464,2,RIGHT,MLO,1,calcification,FINE_LINEAR_BRANCHING,CLUSTERED,0,MALIGNANT,4,Test/P_02464/RIGHT_MLO/scan.nii.gz,Test/P_02464/RIGHT_MLO/roi_calc_1.nii.gz,Test/P_02464/RIGHT_MLO/mask_calc_1.nii.gz
322,P_02498,4,RIGHT,CC,1,calcification,PUNCTATE,CLUSTERED,0,BENIGN,3,Test/P_02498/RIGHT_CC/scan.nii.gz,Test/P_02498/RIGHT_CC/roi_calc_1.nii.gz,Test/P_02498/RIGHT_CC/mask_calc_1.nii.gz
323,P_02498,4,RIGHT,MLO,1,calcification,PUNCTATE,CLUSTERED,0,BENIGN,3,Test/P_02498/RIGHT_MLO/scan.nii.gz,Test/P_02498/RIGHT_MLO/roi_calc_1.nii.gz,Test/P_02498/RIGHT_MLO/mask_calc_1.nii.gz
324,P_02501,3,RIGHT,CC,1,calcification,PLEOMORPHIC,CLUSTERED,0,MALIGNANT,3,Test/P_02501/RIGHT_CC/scan.nii.gz,Test/P_02501/RIGHT_CC/roi_calc_1.nii.gz,Test/P_02501/RIGHT_CC/mask_calc_1.nii.gz


In [11]:
new_csv_file.to_csv('/home/linardos/Datasets/DDSM/processed_calc_case_description_test_set.csv')